In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

In [2]:
path = "../output/kmeans/"
df_each = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        temp = pd.read_csv(f)
        temp = temp[temp.name.apply(lambda x: x != "name")]
        df_each.append((temp, filename))

In [3]:
def score(reduc_time, accuracy, train_time, red_w = 200, ac_w = 1, train_w = 20):
    return (accuracy ** ac_w)*(red_w/int(reduc_time + 1) + train_w/int(train_time + 1))

In [4]:
red_w = 10
ac_w = 10
train_w = 10
included_cols = ["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]

In [5]:
out_df = pd.DataFrame(columns=included_cols)
for res in df_each:
    res_copy = res[0].copy()
    res_copy["filename"] = res[1]
    score_series = res_copy[res_copy.name != "Nothing"][["reduction_time", "train_time", "accuracy"]].apply(lambda x: score(x["reduction_time"], x["accuracy"], red_w, ac_w, train_w), axis=1)
    res_copy_score = res_copy.copy()
    res_copy_score["score_series"] = score_series
    max_ind = res_copy_score.groupby(by=["original_shape"])["score_series"].idxmax()
    collected = res_copy_score.iloc[max_ind][["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]]
    out_df = pd.concat([out_df, collected])

In [10]:
grouped = out_df.groupby(by=["filename"], group_keys=True).apply(lambda a: a[:])
grouped

name  \
filename                                                   
1000_0.33_(-100, 100) 0    extremely sparse JL transform   
                      26   extremely sparse JL transform   
                      68                    JL transform   
                      78   extremely sparse JL transform   
                      122                   JL transform   
...                                                  ...   
500_0_(-500, 500)     116            sparse JL transform   
                      143            sparse JL transform   
                      156  extremely sparse JL transform   
                      197            sparse JL transform   
                      231                   JL transform   

                                        filename  original_shape  \
filename                                                           
1000_0.33_(-100, 100) 0    1000_0.33_(-100, 100)    (1000, 1000)   
                      26   1000_0.33_(-100, 100)   (1000, 10000)   
                      68   1000_0.33_(-100, 100)   (1000, 20000)   
                      78   1000_0.33_(-100, 100)   (10000, 1000)   
                      122  1000_0.33_(-100, 100)  (10000, 10000)   
...                                          ...             ...   
500_0_(-500, 500)     116      500_0_(-500, 500)  (10000, 10000)   
                      143      500_0_(-500, 500)  (10000, 20000)   
                      156      500_0_(-500, 500)   (20000, 1000)   
                      197      500_0_(-500, 500)  (20000, 10000)   
                      231      500_0_(-500, 500)  (20000, 20000)   

                          transformed_shape                    params  \
filename                                                                
1000_0.33_(-100, 100) 0        (1110, 2351)  {'ep': 0.05, 'de': 0.05}   
                      26       (1110, 3135)  {'ep': 0.05, 'de': 0.05}   
                      68       (1110, 2000)  {'ep': 0.05, 'de': 0.05}   
                      78      (10110, 2961)  {'ep': 0.05, 'de': 0.05}   
                      122     (10110, 7189)   {'ep': 0.1, 'de': 0.05}   
...                                     ...                       ...   
500_0_(-500, 500)     116      (10110, 287)   {'ep': 0.5, 'de': 0.05}   
                      143      (10110, 221)    {'ep': 0.5, 'de': 0.1}   
                      156     (20110, 3151)  {'ep': 0.05, 'de': 0.05}   
                      197       (20110, 68)    {'ep': 0.9, 'de': 0.1}   
                      231       (20110, 68)    {'ep': 0.9, 'de': 0.1}   

                           reduction_time  accuracy  train_time  score_series  
filename                                                                       
1000_0.33_(-100, 100) 0          0.016116      0.97    0.638783      8.715012  
                      26         0.131164      0.99    0.765474     10.688152  
                      68         2.809123      0.99    0.450133      4.658938  
                      78         0.375906      0.22    2.095778      0.000003  
                      122       14.604988      1.00    2.349191      2.484848  
...                                   ...       ...         ...           ...  
500_0_(-500, 500)     116        0.879084      1.00    0.275562     11.818182  
                      143        1.876637      1.00    0.330760      6.818182  
                      156        0.688828      1.00    2.663622     11.818182  
                      197        0.917972      0.98    0.267323      9.656315  
                      231        1.733602      1.00    0.281216      6.818182  

[324 rows x 9 columns]

In [11]:
grouped.to_csv("kmeans/kmeans_res_1.csv")

In [8]:
out_df.groupby(["filename", "name"]).count().sort_values(by="original_shape").groupby(level=0).tail(1).sort_values(by="filename")


,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
"1000_0.33_(-100, 100)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-500, 500)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-100, 100)",extremely sparse JL transform,6,6,6,6,6,6,6
"1000_0.66_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-500, 500)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-100, 100)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-1000, 1000)",JL transform,5,5,5,5,5,5,5
"1000_0.99_(-500, 500)",extremely sparse JL transform,3,3,3,3,3,3,3
